#**Qual a finalidade desse notebook?**
Este nootebook tem por fim reunir os dados espalhados, realizar uma primeira avaliação e agregá-los em um dataset, conforme interesse da pesquisa relacionada à compreensão de como certa configuração sócio-econômica favorece, ou não, a realização dos nívies de contanminação e/ou óbitos da relacionados à Covid19

#**Quais são as fonte de dados?**

Este Notebook reune os dados espalhados em um único dataframe. Ao final, ele é exportado num arquivo .zip. Fontes: [IBGE](https://www.ibge.gov.br/estatisticas/downloads-estatisticas.html), [Ministério da Saúde](https://covid.saude.gov.br/), [Atlas Brasil](https://atlasbrasil.org.br) e [INPE](http://labren.ccst.inpe.br/atlas_2017.html)

#**Hands-on ...**

##**1. Preparação**

In [2]:
#################
## Carregar Libs
#################

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from time import sleep
import math

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
###############################
## Variáveis e Funções Globais
###############################

DIR_BASE = "/content/drive/My Drive/Doutorado/Disciplinas/Aprendizado de Maquina/Primeiro Trabalho - ML/database - brasil"

features = ['feature l1', 'feature l2', 'feature l3', 'feature l4']
features_max = ['feature l0', 'feature l1', 'feature l2', 'feature l3', 'feature l4']
features_min = ['feature l0', 'feature l1', 'feature l2']

def isNaN(value):
  return isinstance(value, float) and math.isnan(value)

def show_legend_to(code, df):
  return df.loc[df.loc[df['SIGLA'] == code].index[0]].at['NOME DO INDICADOR']


In [147]:
####################################################
## Seleção de Variáveis e Carregamento de Dataframe
####################################################

_df_AtlasBrasil_Municipios_colunas = ['IBGE7', 'NOME', 'PMATPUB_EF', 
                                      'PMATPUB_EM', 'DIST_EF_PUB', 
                                      'DIST_EM_PUB', 'TTREVA_EF_PUB', 
                                      'TTREVA_EM_PUB', 'IDEB_AI', 'IDEB_AF', 
                                      'DOCSUP_EF_PUB', 'DOCSUP_EM_PUB', 
                                      'TXMOINF', 'TXBRUTAMORT', 'TXCOBPLP', 
                                      'PINTERSAP', 'PINTERDRSAI', 'REN_PIBPC_D', 
                                      'REN_TRPCBF_D','REN_TRPCBPC_D', 
                                      'PDEFAGUA', 'PDEFESGOTO', 'PDEFLIXO', 
                                      'PDEFSAN', 'PPOB_POS', 'PBF', 
                                      'SNIS_PAGUA', 'SNIS_PESGOTO', 
                                      'SNIS_PESGTRA', 'SNIS_PCOLSEL', 
                                      'SNIS_CDI', 'PFOCOS', 'POP_TOT']

colunas_AtlasBrasil_Municipios_PNAD = ['NOME_AGREGA', 'SIGLA_AGREGA', 'ESPVIDA', 
                                       'MORT1', 'T_ENV', 'RAZDEP', 'ANOSEST', 
                                       'T_ANALF15M', 'T_ATRASO_2_FUND', 
                                       'T_ATRASO_2_BASICO', 'T_FUND15A17', 
                                       'R1040', 'RDPC', 'RDPC1', 'RDPC4', 
                                       'RMPOB', 'RPOB', 'PMPOB', 'PPOB', 'GINI',
                                       'POP', 'POPTOT', 'IDHM_E', 'IDHM_L', 
                                       'IDHM_R', 'IDHM']

df_Estados = pd.DataFrame([
  ['ACRE', 'AC'], ['ALAGOAS', 'AL'], ['AMAPÁ', 'AP'], ['AMAZONAS', 'AM'], 
  ['BAHIA', 'BA'], ['CEARÁ', 'CE'], ['DISTRITO FEDERAL', 'DF'],
  ['ESPÍRITO SANTO', 'ES'], ['GOIÁS', 'GO'], ['MARANHÃO', 'MA'], 
  ['MATO GROSSO', 'MT'], ['MATO GROSSO DO SUL', 'MS'], ['MINAS GERAIS', 'MG'], 
  ['PARÁ', 'PA'], ['PARAÍBA', 'PB'], ['PARANÁ', 'PR'], ['PERNAMBUCO', 'PE'], 
  ['PIAUÍ', 'PI'], ['RIO DE JANEIRO', 'RJ'], ['RIO GRANDE DO NORTE', 'RN'], 
  ['RIO GRANDE DO SUL', 'RS'], ['RONDÔNIA', 'RO'], ['RORAIMA', 'RR'], 
  ['SANTA CATARINA', 'SC'], ['SÃO PAULO', 'SP'], ['SERGIPE', 'SE'], 
  ['TOCANTINS', 'TO']], columns=['Estado', 'Sigla'])

## Dados Sociais e Econômicos dos Municípios, usecols=colunas_municipios
_df_AtlasBrasil_Municipios = pd.read_csv(DIR_BASE+'/BASE_MUNICIPIOS_2017.csv', sep=';', usecols=_df_AtlasBrasil_Municipios_colunas)

_df_AtlasBrasil_Municipios_leg = pd.read_csv(DIR_BASE+'/BASE_MUNICIPIOS_2017-LEGENDA.csv', sep=';')

## Dados Sociais e Econômicos das Regiões Metropolitanas (RM)
_df_AtlasBrasil_RM = pd.read_csv(DIR_BASE+'/BASE_RM_2017.csv', sep=';', usecols=_df_AtlasBrasil_Municipios_colunas)

## Dados Sociais da PNAD/IDHM por Município  -  , usecols=colunas_PNAD
_df_AtlasBrasil_PNAD_RM = pd.read_csv(DIR_BASE+'/BASE_PNAD_2017.csv', sep=';', usecols=colunas_AtlasBrasil_Municipios_PNAD)

_df_AtlasBrasil_PNAD_RM_leg = pd.read_csv(DIR_BASE+'/BASE_PNAD_2017-LEGENDA.csv', sep=';')

## Dados Sociais da PNAD/IDHM por Município  -  , usecols=colunas_PNAD
colunas_COVID = ['estado',	'municipio', 	'codmun',	'data', 'casosAcumulado',	'obitosAcumulado', 'interior/metropolitana']
_df_MinDaSaude_COVID = pd.read_csv(DIR_BASE+'/HIST_PAINEL_COVIDBR_04nov2020.csv', sep=';', usecols=colunas_COVID)

## Dados simples dos municipios do Brasil segundo IBGE (contem o codigo oficial do municipio)
_df_IBGE_Municipios_colunas = ['IBGE', 'IBGE7', 'UF', 'Município', 'Região', 'Porte', 'Capital']
_df_IBGE_Municipios = pd.read_csv(DIR_BASE+'/Lista_Municípios_com_IBGE_Brasil_Versao_CSV.csv', sep=';', encoding='latin1', usecols=_df_IBGE_Municipios_colunas)

## Dados de Luminosidade Média
_df_INPE_Muncipios_luzdifusa = pd.read_csv(DIR_BASE+'/diffuse_means_sedes-munic.csv', sep=';')

## Dados de Luminosidade Direta
_df_INPE_Muncipios_luzdireta = pd.read_csv(DIR_BASE+'/direct_normal_means_sedes-munic.csv', sep=';')


In [148]:
#########################
## Ajustes - AtlasBrasil - 01
#########################

df_AtlasBrasil_Municipios = _df_AtlasBrasil_Municipios.replace(to_replace='&apos;', value='\'', regex=True)
df_AtlasBrasil_Municipios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   IBGE7          5570 non-null   int64  
 1   NOME           5570 non-null   object 
 2   PMATPUB_EF     5570 non-null   object 
 3   PMATPUB_EM     5561 non-null   object 
 4   DIST_EF_PUB    5570 non-null   object 
 5   DIST_EM_PUB    5560 non-null   object 
 6   TTREVA_EF_PUB  0 non-null      float64
 7   TTREVA_EM_PUB  0 non-null      float64
 8   IDEB_AI        5476 non-null   object 
 9   IDEB_AF        5462 non-null   object 
 10  DOCSUP_EF_PUB  5570 non-null   object 
 11  DOCSUP_EM_PUB  5561 non-null   object 
 12  TXMOINF        5429 non-null   object 
 13  TXBRUTAMORT    5568 non-null   object 
 14  TXCOBPLP       5558 non-null   object 
 15  PINTERSAP      5570 non-null   object 
 16  PINTERDRSAI    5570 non-null   object 
 17  REN_PIBPC_D    0 non-null      float64
 18  REN_TRPC

In [149]:
#########################
## Ajustes - AtlasBrasil - 02
#########################

_df = df_AtlasBrasil_Municipios.copy()
_df.drop(columns=[
    'TTREVA_EF_PUB', 
    'TTREVA_EM_PUB', 
    'REN_PIBPC_D', 
    'SNIS_PCOLSEL',
    'SNIS_PESGOTO',
    'SNIS_PESGTRA',
    'SNIS_CDI'], inplace=True)
print(_df.shape[0])
_df.dropna(inplace=True)
print(_df.shape[0])


5570
4663


In [150]:
#########################
## Ajustes - AtlasBrasil - 03
#########################

df_AtlasBrasil_Municipios = _df
df_AtlasBrasil_Municipios.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4663 entries, 0 to 5569
Data columns (total 26 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   IBGE7          4663 non-null   int64 
 1   NOME           4663 non-null   object
 2   PMATPUB_EF     4663 non-null   object
 3   PMATPUB_EM     4663 non-null   object
 4   DIST_EF_PUB    4663 non-null   object
 5   DIST_EM_PUB    4663 non-null   object
 6   IDEB_AI        4663 non-null   object
 7   IDEB_AF        4663 non-null   object
 8   DOCSUP_EF_PUB  4663 non-null   object
 9   DOCSUP_EM_PUB  4663 non-null   object
 10  TXMOINF        4663 non-null   object
 11  TXBRUTAMORT    4663 non-null   object
 12  TXCOBPLP       4663 non-null   object
 13  PINTERSAP      4663 non-null   object
 14  PINTERDRSAI    4663 non-null   object
 15  REN_TRPCBF_D   4663 non-null   object
 16  REN_TRPCBPC_D  4663 non-null   object
 17  PDEFAGUA       4663 non-null   object
 18  PDEFESGOTO     4663 non-null

In [151]:
##########################
## Ajustes - Min da Saúde - 01
##########################

## Ajustar NaN e tipo em colunas/linhas (reduzir tamanho da base em função de NaNs
df_MinDaSaude_COVID = _df_MinDaSaude_COVID[~_df_MinDaSaude_COVID['municipio'].isnull()]

## Modificar o tipo da variável 'codmun'
df_MinDaSaude_COVID = df_MinDaSaude_COVID.astype({ 'codmun': int }) 

## Selecionar data (o alvo será a qnt de casos acumulados município a município em 01-11-2020)
df_MinDaSaude_COVID = df_MinDaSaude_COVID[df_MinDaSaude_COVID['data'] == '2020-11-01']
df_MinDaSaude_COVID.info()

## Nada a fazer

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 7554 to 1253901
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   estado                  5570 non-null   object 
 1   municipio               5570 non-null   object 
 2   codmun                  5570 non-null   int64  
 3   data                    5570 non-null   object 
 4   casosAcumulado          5570 non-null   int64  
 5   obitosAcumulado         5570 non-null   int64  
 6   interior/metropolitana  5570 non-null   float64
dtypes: float64(1), int64(3), object(3)
memory usage: 348.1+ KB


In [152]:
#######################################################
## Avaliando NaNs em variáveis do dataframe do PNAD_RM - 01
#######################################################

df_AtlasBrasil_PNAD_RM = _df_AtlasBrasil_PNAD_RM.copy()
df_AtlasBrasil_PNAD_RM.info()

## Nada a fazer

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49 entries, 0 to 48
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   NOME_AGREGA        49 non-null     object
 1   SIGLA_AGREGA       49 non-null     object
 2   ESPVIDA            49 non-null     object
 3   MORT1              49 non-null     object
 4   T_ENV              49 non-null     object
 5   RAZDEP             49 non-null     object
 6   ANOSEST            49 non-null     object
 7   T_ANALF15M         49 non-null     object
 8   T_ATRASO_2_BASICO  49 non-null     object
 9   T_ATRASO_2_FUND    49 non-null     object
 10  T_FUND15A17        49 non-null     object
 11  R1040              49 non-null     object
 12  RDPC               49 non-null     object
 13  RDPC1              49 non-null     object
 14  RDPC4              49 non-null     object
 15  RMPOB              49 non-null     object
 16  RPOB               49 non-null     object
 17 

In [154]:
#########################################
## Divergência de formato de cod do IBGE 
#########################################

print(df_MinDaSaude_COVID['codmun'].head())
print(df_AtlasBrasil_Municipios['IBGE7'].head())
print(_df_IBGE_Municipios[['IBGE', 'IBGE7']].head())

7554    110001
7777    110002
8000    110003
8223    110004
8446    110005
Name: codmun, dtype: int64
0    1100015
1    1100023
2    1100031
3    1100049
4    1100056
Name: IBGE7, dtype: int64
     IBGE    IBGE7
0  110001  1100015
1  110002  1100023
2  110003  1100031
3  110004  1100049
4  110005  1100056


In [155]:
##############################################
## Avaliando divergências entre os dataframes - 01
##############################################

municipios = df_MinDaSaude_COVID['municipio']
nomes = df_AtlasBrasil_Municipios['NOME']

print(municipios.shape[0], municipios.isin(nomes).sum())
print(nomes.shape[0], nomes.isin(municipios).sum())

5570 4704
4663 4641


In [156]:
#######################################################################
## Ajustar divergências entre dataframes (base: cod do IBGE do Munic) - 01
#######################################################################

df_IBGE_Municipios = _df_IBGE_Municipios.copy()

IBGEs = df_IBGE_Municipios.loc[:,'IBGE'];
IBGE7s = df_IBGE_Municipios.loc[:,'IBGE7'];

# pode ser mais interessante anexada a coluna divergente
for index in range(len(IBGE7s)):
  df_AtlasBrasil_Municipios.replace(to_replace=IBGE7s[index], value=IBGEs[index], regex=True, inplace=True)


In [157]:
###############################
## Ajustar nomes das variaveis - 01
###############################

df_AtlasBrasil_Municipios.rename(columns={"IBGE7": "IBGE"}, inplace=True)
df_MinDaSaude_COVID.rename(columns={"codmun": "IBGE"}, inplace=True)

In [158]:
###############################################################################
## Ajustar labels para a variável Capital no dataframe Lista de Munic. do IBGE
###############################################################################

df_IBGE_Municipios.loc[df_IBGE_Municipios['Capital'] != 'Capital', 'Capital'] = 'Interior'
df_IBGE_Municipios['Capital'].unique()

array(['Interior', 'Capital'], dtype=object)

##**2. Reunindo os dataframes em um único - *Merge***

### 2.3 Primeiro Merge

> (dataframe do) *Atlas Brasil* com (dataframe do) *Min. da Saúde*

In [159]:
df_AtlasBrasil_Municipios.drop(columns=['NOME'], inplace=True)

# Merge entre os dataframes

dataset = pd.merge(df_AtlasBrasil_Municipios, df_MinDaSaude_COVID, on=['IBGE'], how='inner')

### 2.4 Segundo Merge

> (dataframe gerado pelo) *primeiro merge* com (dataframe da) *Lista de Municipios IBGE*

In [160]:
#Incluindo features (Porte e Capital) de Lista_Municípios_com_IBGE_Brasil

df_IBGE_Municipios_reduzido = df_IBGE_Municipios.drop(columns=['Município', 'IBGE7', 'UF']) 
print(df_IBGE_Municipios_reduzido.columns)

dataset = pd.merge(dataset, df_IBGE_Municipios_reduzido, on=['IBGE'], how='inner')

Index(['IBGE', 'Região', 'Porte', 'Capital'], dtype='object')


### 2.5 Terceiro Merge

> (dataframe gerado pelo) *segundo merge* com (dataframe do) *PNAD (IBGE)*

In [161]:
df_AtlasBrasil_PNAD_RM = df_AtlasBrasil_PNAD_RM.rename(columns={"SIGLA_AGREGA": "estado"})
df_AtlasBrasil_PNAD_RM.drop(columns=['NOME_AGREGA'], inplace=True)
df_AtlasBrasil_PNAD_RM.drop([0], inplace=True)
df_AtlasBrasil_PNAD_RM.drop(np.arange(start=28, stop=49, step=1), inplace=True)

dataset = pd.merge(dataset, df_AtlasBrasil_PNAD_RM, on=['estado'], how='inner')

###2.6 Quarto Merge

In [162]:
## Ajustando dados de Luz Direta para os Muncipios
for index in range(len(df_Estados)):
  _df_INPE_Muncipios_luzdireta.replace(df_Estados.loc[index,'Estado'], df_Estados.loc[index,'Sigla'], inplace=True)

_df_INPE_Muncipios_luzdireta.rename(columns={"NAME": "municipio"}, inplace=True)
_df_INPE_Muncipios_luzdireta.rename(columns={"STATE": "estado"}, inplace=True)
_df_INPE_Muncipios_luzdireta.drop(columns=['CLASS', 'ID'], inplace=True)

dataset = pd.merge(dataset, _df_INPE_Muncipios_luzdireta, on=['municipio', 'estado'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4643 entries, 0 to 4642
Data columns (total 73 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IBGE                    4643 non-null   int64  
 1   PMATPUB_EF              4643 non-null   object 
 2   PMATPUB_EM              4643 non-null   object 
 3   DIST_EF_PUB             4643 non-null   object 
 4   DIST_EM_PUB             4643 non-null   object 
 5   IDEB_AI                 4643 non-null   object 
 6   IDEB_AF                 4643 non-null   object 
 7   DOCSUP_EF_PUB           4643 non-null   object 
 8   DOCSUP_EM_PUB           4643 non-null   object 
 9   TXMOINF                 4643 non-null   object 
 10  TXBRUTAMORT             4643 non-null   object 
 11  TXCOBPLP                4643 non-null   object 
 12  PINTERSAP               4643 non-null   object 
 13  PINTERDRSAI             4643 non-null   object 
 14  REN_TRPCBF_D            4643 non-null   

## 3. Exportando o **dataset**

In [186]:
# ## Exportar o resultado para evitar reconstruir o dataset
compression_opts = dict(method='zip', archive_name='dataset.csv')  
dataset.to_csv (DIR_BASE+'/dataset.zip', sep=';', index=False, header=True, compression=compression_opts)

======================================================================
#Outras fontes de dados a serem investigadas
Não serão agregadas ao dataset anteriormente construído. 

#1. Dados Completos de Estudos Sócio-econômico-político dos Município apenas do IBGE

Recuperados dados dos setores de *trabalho*, *saúde*, *educação* e *segurança alimentar* para formarem dataframes

In [163]:
# TODO: ## Refazer parte do trabalho para adicionar as features do PNAD <<<<<<<<<
# TODO: ## Refazer parte do trabalho para adicionar demais features espalhadas (base IBGE: porte, capital) <<<<<<<<<

# ## Dados socio economicos - TRABALHO & INCLUSAO PRODUTIVA - dos municipios IBGE - 2017
PATH = '/content/drive/My Drive/toColab/Disciplinas/Machine Learning/Dados'
df_BM_trabalho_leg = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_legenda-trabalho_v0.csv', sep=';')

df_BM_trabalho_columns = ['Cod Municipio', 'MTIP03', 'MTIP05', 
                          'MTIP07', 'MTIP09', 'MTIP11', 'MTIP13']
df_BM_trabalho = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_trabalho_v0.csv', sep=';', usecols=df_BM_trabalho_columns)


# ## Dados socio economicos - SEGURANCA ALIMENTAR - dos municipios IBGE - 2017
df_BM_segalimentar_leg = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_legenda-segalimentar_v0.csv', sep=';')

df_BM_segalimentar_columns = ['Cod Municipio', 'MSAN042B', 'MSAN01', 'MSAN04', 
                              'MSAN042B', 'MSAN05', 'MSAN16', 'MSAN171', 
                              'MSAN173', 'MSAN174', 'MSAN177', 'MSAN1710', 
                              'MSAN1712', 'MSAN1713']
# filtro do ano: MSAN042B 
df_BM_segalimentar = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_segalimentar_v0.csv', sep=';', usecols=df_BM_segalimentar_columns)


# ## Dados socio economicos - SAUDE - dos municipios IBGE - 2017
df_BM_saude_leg = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_legenda-saude_v0.csv', sep=';')

df_BM_saude_columns = ['Cod Municipio', 'MSAU181', 'MSAU01', 'MSAU19', 'MSAU20', 'MSAU201', 
                       'MSAU21', 'MSAU25', 'MSAU28', 'MSAU281', 'MSAU29', 
                       'MSAU2911', 'MSAU2914', 'MSAU30', 'MSAU3011', 'MSAU31', 
                       'MSAU3111', 'MSAU325', 'MSAU326', 'MSAU33', 'MSAU34', 
                       'MSAU42', 'MSAU43', 'MSAU43', 'MSAU43', 'MSAU43', 
                       'MSAU44', 'MSAU456', 'MSAU451', 'MSAU452', 'MSAU453', 
                       'MSAU454', 'MSAU47', 'MSAU48', 'MSAU51']
# filtro do ano: MSAU181
df_BM_saude = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_saude_v0.csv', sep=';', usecols=df_BM_saude_columns)


# ## Dados socio economicos - EDUCACAO - dos municipios IBGE - 2017
df_BM_educacao_leg = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_legenda-educacao_v0.csv', sep=';')

df_BM_educacao_columns = ['Cod Municipio', 'MEDU151', 'MEDU01', 'MEDU111', 'MEDU113', 
                          'MEDU115', 'MEDU116', 'MEDU117', 'MEDU119', 'MEDU1110', 
                          'MEDU1112', 'MEDU1113', 'MEDU1114', 'MEDU1115', 
                          'MEDU1116', 'MEDU1117', 'MEDU1118', 'MEDU1119', 
                          'MEDU121', 'MEDU122', 'MEDU123', 'MEDU125', 'MEDU16', 
                          'MEDU161b', 'MEDU21', 'MEDU211B', 'MEDU48', 'MEDU501', 
                          'MEDU502', 'MEDU503', 'MEDU504', 'MEDU52', 'MEDU521']
# filtro do ano: MEDU151
df_BM_educacao = pd.read_csv(PATH+'/Base_MUNIC_2018_ods_20201103_reduzida_v0_educacao_v0.csv', sep=';', usecols=df_BM_educacao_columns)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (82,85,88) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (48) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 1.1. Preparando dataframes para merge

In [31]:
# df_BM_trabalho.info()

In [32]:
# df_BM_segalimentar.info()

In [33]:
# df_BM_saude.dropna()

In [34]:
# df_BM_educacao.info()

In [29]:
# ## Extraindo os dados de 2017 - ano mais recente da medição
# df_BM_trabalho = df_BM_trabalho[df_BM_trabalho['MTIP011'] == '2017']
# df_BM_segalimentar = df_BM_segalimentar[df_BM_segalimentar['MSAN042B'] == '2017'] 
# df_BM_saude = df_BM_saude[df_BM_saude['MSAU181'] == '2017'] 
# df_BM_educacao = df_BM_educacao[df_BM_educacao['MEDU151'] == '2017'] 

In [164]:
# ## Sincronizar codigos dos municipios - transformar de formato ibge7 para formato ibge
for index in range(len(IBGE7s)):
  df_BM_trabalho.replace(to_replace=IBGE7s[index], value=IBGEs[index], regex=True, inplace=True)

# ## Renomear coluna para executar merge
df_BM_trabalho.rename(columns={"Cod Municipio": "IBGE"}, inplace=True)


### 1.1.1 Deinindo melhores features

In [165]:
# Avalia as colunas que possuem a menor quantidade de NaN (100) IBGE Trabalho

print(df_BM_trabalho.info())
print(df_BM_trabalho.head())

# tolerance = np.round(_df.shape[0]*0.05, 0).astype(int);
# print('# Tolerance: ', tolerance)
# _df_good_columns = []
# for column in _df.columns:
#   if (_df[_df[column].isna()].shape[0] < tolerance):
#     _df_good_columns.append(column)

# for column in _df_good_columns:
#   nans = _df[_df[column].isna()].shape[0];
#   try:
#     s = _df_leg.loc[_df_leg.loc[_df_leg['value'] == column].index[0]]
#     print(column, ': ', s['feature l0'], ' > ', s['feature l1'], ' > ', s['feature l2'], ' > ', s['feature l3'], ' > ', s['feature l4'], ' - ', nans, ' NaNs')
#   except:
#     print(column, 'Error')

# print('# Shape after: ', _df[_df_good_columns].shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 7 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   IBGE    5570 non-null   int64 
 1   MTIP03  5570 non-null   object
 2   MTIP05  5570 non-null   object
 3   MTIP07  5570 non-null   object
 4   MTIP09  5570 non-null   object
 5   MTIP11  5570 non-null   object
 6   MTIP13  5570 non-null   object
dtypes: int64(1), object(6)
memory usage: 304.7+ KB
None
     IBGE MTIP03  ...             MTIP11             MTIP13
0  110001    Sim  ...                Não                Não
1  110002    Sim  ...                Sim                Sim
2  110003    Sim  ...                Não                Sim
3  110004    Sim  ...  Não sabe informar  Não sabe informar
4  110005    Sim  ...                Sim                Sim

[5 rows x 7 columns]


In [166]:
# ## Sincronizar codigos dos municipios - transformar de formato ibge7 para formato ibge
for index in range(len(IBGE7s)):
  df_BM_saude.replace(to_replace=IBGE7s[index], value=IBGEs[index], regex=True, inplace=True)

# ## Renomear coluna para executar merge
df_BM_saude.rename(columns={"Cod Municipio": "IBGE"}, inplace=True)
df_BM_saude.replace(to_replace='-', value=np.nan, inplace=True)


In [167]:
# Avalia as colunas que possuem a menor quantidade de NaN (100) IBGE Saude

df_BM_saude.drop(columns=['MSAU281', 'MSAU3111', 'MSAU325', 'MSAU326'], inplace=True)
print(df_BM_saude.info())
print(df_BM_saude.head())

# tolerance = np.round(_df.shape[0]*0.05, 0).astype(int);
# print('# Tolerance: ', tolerance)
# _df_good_columns = []
# for column in _df.columns:
#   if (_df[_df[column].isna()].shape[0] < tolerance):
#     _df_good_columns.append(column)

# for column in _df_good_columns:
#   nans = _df[_df[column].isna()].shape[0];
#   try:
#     s = _df_leg.loc[_df_leg.loc[_df_leg['value'] == column].index[0]]
#     print(column, ': ', s['feature l0'], ' > ', s['feature l1'], ' > ', s['feature l2'], ' > ', s['feature l3'], ' > ', s['feature l4'], ' - ', nans, ' NaNs')
#   except:
#     print(column, 'Error')

# print('# Shape after: ', _df[_df_good_columns].shape)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 28 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IBGE      5570 non-null   int64 
 1   MSAU01    5570 non-null   object
 2   MSAU181   5341 non-null   object
 3   MSAU19    5570 non-null   object
 4   MSAU20    5570 non-null   object
 5   MSAU201   5496 non-null   object
 6   MSAU21    5496 non-null   object
 7   MSAU25    5570 non-null   object
 8   MSAU28    5570 non-null   object
 9   MSAU29    5570 non-null   object
 10  MSAU2911  5474 non-null   object
 11  MSAU2914  5474 non-null   object
 12  MSAU30    5474 non-null   object
 13  MSAU3011  5204 non-null   object
 14  MSAU31    5570 non-null   object
 15  MSAU33    5570 non-null   object
 16  MSAU34    5559 non-null   object
 17  MSAU42    5570 non-null   object
 18  MSAU43    5570 non-null   object
 19  MSAU44    5570 non-null   object
 20  MSAU456   5570 non-null   object
 21  MSAU451   5121

In [168]:
# ## Sincronizar codigos dos municipios - transformar de formato ibge7 para formato ibge
for index in range(len(IBGE7s)):
  df_BM_educacao.replace(to_replace=IBGE7s[index], value=IBGEs[index], regex=True, inplace=True)

# ## Renomear coluna para executar merge
df_BM_educacao.rename(columns={"Cod Municipio": "IBGE"}, inplace=True)
df_BM_educacao.replace(to_replace='-', value=np.nan, inplace=True)

In [169]:
# Avalia as colunas que possuem a menor quantidade de NaN (100) IBGE Educacao

df_BM_educacao.drop(columns=['MEDU1119', 'MEDU125', 'MEDU151', 'MEDU211B', 
                             'MEDU501', 'MEDU502', 'MEDU503', 'MEDU504',
                             'MEDU121', 'MEDU121', 'MEDU122', 'MEDU123',
                             'MEDU521'], inplace=True)
print(df_BM_educacao.info())
print(df_BM_educacao.head())

# tolerance = np.round(_df.shape[0]*0.05, 0).astype(int);
# print('# Tolerance: ', tolerance)
# _df_good_columns = []
# for column in _df.columns:
#   if (_df[_df[column].isna()].shape[0] < tolerance):
#     _df_good_columns.append(column)

# for column in _df_good_columns:
#   nans = _df[_df[column].isna()].shape[0];
#   try:
#     s = _df_leg.loc[_df_leg.loc[_df_leg['value'] == column].index[0]]
#     print(column, ': ', s['feature l0'], ' > ', s['feature l1'], ' > ', s['feature l2'], ' > ', s['feature l3'], ' > ', s['feature l4'], ' - ', nans, ' NaNs')
#   except:
#     print(column, 'Error')

# print('# Shape after: ', _df[_df_good_columns].shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 21 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IBGE      5570 non-null   int64 
 1   MEDU01    5570 non-null   object
 2   MEDU111   5561 non-null   object
 3   MEDU113   5561 non-null   object
 4   MEDU115   5561 non-null   object
 5   MEDU116   5561 non-null   object
 6   MEDU117   5561 non-null   object
 7   MEDU119   5561 non-null   object
 8   MEDU1110  5561 non-null   object
 9   MEDU1112  5561 non-null   object
 10  MEDU1113  5561 non-null   object
 11  MEDU1114  5561 non-null   object
 12  MEDU1115  5561 non-null   object
 13  MEDU1116  5561 non-null   object
 14  MEDU1117  5561 non-null   object
 15  MEDU1118  5561 non-null   object
 16  MEDU16    5570 non-null   object
 17  MEDU161b  5332 non-null   object
 18  MEDU21    5570 non-null   object
 19  MEDU48    5570 non-null   object
 20  MEDU52    5570 non-null   object
dtypes: int64(1), o

In [170]:
# ## Sincronizar codigos dos municipios - transformar de formato ibge7 para formato ibge
for index in range(len(IBGE7s)):
  df_BM_segalimentar.replace(to_replace=IBGE7s[index], value=IBGEs[index], regex=True, inplace=True)

# ## Renomear coluna para executar merge
df_BM_segalimentar.rename(columns={"Cod Municipio": "IBGE"}, inplace=True)
df_BM_segalimentar.replace(to_replace='-', value=np.nan, inplace=True)

In [171]:
# Avalia as colunas que possuem a menor quantidade de NaN (100) IBGE Educacao

df_BM_segalimentar.drop(columns=['MSAN042B'], inplace=True)
print(df_BM_segalimentar.info())
print(df_BM_segalimentar.head())

# tolerance = np.round(_df.shape[0]*0.05, 0).astype(int);
# print('# Tolerance: ', tolerance)
# _df_good_columns = []
# for column in _df.columns:
#   if (_df[_df[column].isna()].shape[0] < tolerance):
#     _df_good_columns.append(column)

# for column in _df_good_columns:
#   nans = _df[_df[column].isna()].shape[0];
#   try:
#     s = _df_leg.loc[_df_leg.loc[_df_leg['value'] == column].index[0]]
#     print(column, ': ', s['feature l0'], ' > ', s['feature l1'], ' > ', s['feature l2'], ' > ', s['feature l3'], ' > ', s['feature l4'], ' - ', nans, ' NaNs')
#   except:
#     print(column, 'Error')

# print('# Shape after: ', _df[_df_good_columns].shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5570 entries, 0 to 5569
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IBGE      5570 non-null   int64 
 1   MSAN01    5570 non-null   object
 2   MSAN04    5570 non-null   object
 3   MSAN05    5570 non-null   object
 4   MSAN16    5570 non-null   object
 5   MSAN171   5570 non-null   object
 6   MSAN173   5570 non-null   object
 7   MSAN174   5570 non-null   object
 8   MSAN177   5570 non-null   object
 9   MSAN1710  5570 non-null   object
 10  MSAN1712  5570 non-null   object
 11  MSAN1713  5570 non-null   object
dtypes: int64(1), object(11)
memory usage: 522.3+ KB
None
     IBGE                                             MSAN01  ... MSAN1712 MSAN1713
0  110001                               Não possui estrutura  ...      Sim      Sim
1  110002  Secretaria em conjunto com outras políticas se...  ...      Sim      Sim
2  110003                               Não possui 

## 1.2. Merges dos dataframes anteriores

### *Importante*: tem como base a tabela de municípios do IBGE: Lista_Municípios_com_IBGE_Brasil

In [185]:
# ## Merge entre a dataframes
# ex_dataset = pd.merge(dataset, df_BM_trabalho, on=['IBGE'], how='inner')

ibge_dataset = pd.merge(df_BM_trabalho, df_BM_segalimentar, on=['IBGE'], how='inner')
ibge_dataset = pd.merge(ibge_dataset, df_BM_saude, on=['IBGE'], how='inner')
ibge_dataset = pd.merge(ibge_dataset, df_BM_educacao, on=['IBGE'], how='inner')

print(ibge_dataset.info())
print(ibge_dataset.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 66 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IBGE      5570 non-null   int64 
 1   MTIP03    5570 non-null   object
 2   MTIP05    5570 non-null   object
 3   MTIP07    5570 non-null   object
 4   MTIP09    5570 non-null   object
 5   MTIP11    5570 non-null   object
 6   MTIP13    5570 non-null   object
 7   MSAN01    5570 non-null   object
 8   MSAN04    5570 non-null   object
 9   MSAN05    5570 non-null   object
 10  MSAN16    5570 non-null   object
 11  MSAN171   5570 non-null   object
 12  MSAN173   5570 non-null   object
 13  MSAN174   5570 non-null   object
 14  MSAN177   5570 non-null   object
 15  MSAN1710  5570 non-null   object
 16  MSAN1712  5570 non-null   object
 17  MSAN1713  5570 non-null   object
 18  MSAU01    5570 non-null   object
 19  MSAU181   5341 non-null   object
 20  MSAU19    5570 non-null   object
 21  MSAU20    5570

In [186]:
if ('estado' in ibge_dataset.columns): 
  ibge_dataset.drop(columns=['estado'], inplace=True)
ibge_dataset = ex_dataset.assign(estado="")

for index in range(len(IBGEs)):
  UF = df_IBGE_Municipios.loc[df_IBGE_Municipios[df_IBGE_Municipios['IBGE'] == IBGEs[index]]['UF'].index[0], 'UF']
  ibge_dataset.loc[ibge_dataset['IBGE'] == IBGEs[index], 'estado'] = UF

ibge_dataset.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 5570 entries, 0 to 5569
Data columns (total 66 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   IBGE      5570 non-null   int64 
 1   MTIP03    5570 non-null   object
 2   MTIP05    5570 non-null   object
 3   MTIP07    5570 non-null   object
 4   MTIP09    5570 non-null   object
 5   MTIP11    5570 non-null   object
 6   MTIP13    5570 non-null   object
 7   MSAN01    5570 non-null   object
 8   MSAN04    5570 non-null   object
 9   MSAN05    5570 non-null   object
 10  MSAN16    5570 non-null   object
 11  MSAN171   5570 non-null   object
 12  MSAN173   5570 non-null   object
 13  MSAN174   5570 non-null   object
 14  MSAN177   5570 non-null   object
 15  MSAN1710  5570 non-null   object
 16  MSAN1712  5570 non-null   object
 17  MSAN1713  5570 non-null   object
 18  MSAU01    5570 non-null   object
 19  MSAU181   5341 non-null   object
 20  MSAU19    5570 non-null   object
 21  MSAU20    5570

In [187]:
ibge_dataset_ex = ibge_dataset.copy()

ibge_dataset_ex = pd.merge(ibge_dataset_ex, df_IBGE_Municipios_reduzido, on=['IBGE'], how='inner')
ibge_dataset_ex = pd.merge(ibge_dataset_ex, df_AtlasBrasil_Municipios, on=['IBGE'], how='inner')
ibge_dataset_ex = pd.merge(ibge_dataset_ex, df_MinDaSaude_COVID, on=['IBGE'], how='inner')
ibge_dataset_ex.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4663 entries, 0 to 4662
Data columns (total 99 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   IBGE                    4663 non-null   int64  
 1   MTIP03                  4663 non-null   object 
 2   MTIP05                  4663 non-null   object 
 3   MTIP07                  4663 non-null   object 
 4   MTIP09                  4663 non-null   object 
 5   MTIP11                  4663 non-null   object 
 6   MTIP13                  4663 non-null   object 
 7   MSAN01                  4663 non-null   object 
 8   MSAN04                  4663 non-null   object 
 9   MSAN05                  4663 non-null   object 
 10  MSAN16                  4663 non-null   object 
 11  MSAN171                 4663 non-null   object 
 12  MSAN173                 4663 non-null   object 
 13  MSAN174                 4663 non-null   object 
 14  MSAN177                 4663 non-null   

In [198]:
ibge_dataset_ex.drop(columns=['estado_y'], inplace=True)
ibge_dataset_ex.rename(columns={"estado_x": "estado"}, inplace=True)

In [199]:
# ## Exportar o resultado para evitar reconstruir o dataset
compression_opts = dict(method='zip', archive_name='2018_ibge_dataset_ex.csv')  
ibge_dataset_ex.to_csv(DIR_BASE+'/2018_ibge_dataset_ex.zip', sep=';', index=False, header=True, compression=compression_opts)

# Verificações da Base de Dados de Rodrigo - Municipios

In [ ]:
# ## Base de Municipios de Rodrigo
df_muncipios_brasileiros = pd.read_csv(DIR_BASE+'/municipios_brasileiros.csv', sep=',')
df_muncipios_brasileiros

,Código IBGE,Nome do Município,Código UF,UF,Estado,Latitude,Longitude
0,5200050,Abadia de Goiás,52,GO,Goiás,-16.75730,-49.4412
1,3100104,Abadia dos Dourados,31,MG,Minas Gerais,-18.48310,-47.3916
2,5200100,Abadiânia,52,GO,Goiás,-16.19700,-48.7057
3,3100203,Abaeté,31,MG,Minas Gerais,-19.15510,-45.4444
4,1500107,Abaetetuba,15,PA,Pará,-1.72183,-48.8788
...,...,...,...,...,...,...,...
5565,4314548,Pinto Bandeira,43,RS,Rio Grande do Sul,-29.09750,-51.4503
5566,4220000,Balneário Rincão,42,SC,Santa Catarina,-28.83140,-49.2352
5567,4212650,Pescaria Brava,42,SC,Santa Catarina,-28.39660,-48.8864
5568,1504752,Mojuí dos Campos,15,PA,Pará,-2.68220,-54.6425


In [ ]:
_df_Municipios

,ConcatUF+Mun,IBGE,IBGE7,UF,Município,Região,População 2010,Porte,Capital
0,ROAlta Floresta D'oeste,110001,1100015,RO,Alta Floresta D´oeste,Região Norte,24392.0,Pequeno II,Interior
1,ROAriquemes,110002,1100023,RO,Ariquemes,Região Norte,90353.0,Médio,Interior
2,ROCabixi,110003,1100031,RO,Cabixi,Região Norte,6313.0,Pequeno I,Interior
3,ROCacoal,110004,1100049,RO,Cacoal,Região Norte,78574.0,Médio,Interior
4,ROCerejeiras,110005,1100056,RO,Cerejeiras,Região Norte,17029.0,Pequeno I,Interior
...,...,...,...,...,...,...,...,...,...
5565,GOVianópolis,522200,5222005,GO,Vianópolis,Região Centro-Oeste,12548.0,Pequeno I,Interior
5566,GOVicentinópolis,522205,5222054,GO,Vicentinópolis,Região Centro-Oeste,7371.0,Pequeno I,Interior
5567,GOVila Boa,522220,5222203,GO,Vila Boa,Região Centro-Oeste,4735.0,Pequeno I,Interior
5568,GOVila Propício,522230,5222302,GO,Vila Propício,Região Centro-Oeste,5145.0,Pequeno I,Interior


In [ ]:

print(df_muncipios_brasileiros['Código IBGE'].shape)
print(_df_Municipios['IBGE7'].shape)

print(_df_Municipios[~_df_Municipios['IBGE7'].isin(df_muncipios_brasileiros['Código IBGE'])].shape)
print(df_muncipios_brasileiros[~df_muncipios_brasileiros['Código IBGE'].isin(_df_Municipios['IBGE7'])].shape)

(5570,)
(5570,)
Int64Index([], dtype='int64')
(0, 7)


In [ ]:
# ## Dados de Luminosidade Média
df_muncipios_luzdifusa = pd.read_csv(DIR_BASE+'/diffuse_means_sedes-munic.csv', sep=';')
print(df_muncipios_luzdifusa.columns)
print(df_muncipios_luzdifusa['NAME'])

# ## Dados de Luminosidade Direta
df_muncipios_luzdireta = pd.read_csv(DIR_BASE+'/direct_normal_means_sedes-munic.csv', sep=';')
print(df_muncipios_luzdireta.columns)
print(df_muncipios_luzdireta['NAME'])

Index(['ID', 'LON', 'LAT', 'NAME', 'CLASS', 'STATE', 'ANNUAL', 'JAN', 'FEB',
       'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'],
      dtype='object')
0                        Brasiléia
1                   Epitaciolândia
2                     Assis Brasil
3                           Xapuri
4                         Capixaba
                   ...            
5564                   Praia Norte
5565                       Sampaio
5566    São Sebastião do Tocantins
5567           Buriti do Tocantins
5568               Carrasco Bonito
Name: NAME, Length: 5569, dtype: object
Index(['ID', 'LON', 'LAT', 'NAME', 'CLASS', 'STATE', 'ANNUAL', 'JAN', 'FEB',
       'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC'],
      dtype='object')
0                        Brasiléia
1                   Epitaciolândia
2                     Assis Brasil
3                           Xapuri
4                         Capixaba
                   ...            
5564       